# Functionality

> Only development notebook where the goal is to save the ranking in a csv file only (analysis will be done on evaluation.ipynb notebook"

1. The user provides a prompt containing that ask the LM to rank for n sentences with the ideal sentence based on similarity.

2. Save the response (as dataframe) from the model and extract the rank from the free text generated by the model and save it into the same dataframe as a new column

3. Creating a column that has the correlation for each prompt with the gold rank (here you provide the gold rank)

In [ ]:
import re
import pandas as pd
from scipy import stats
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import statistics
import ipywidgets as widgets
import os
import csv
import datetime
import openai
from dotenv import load_dotenv
import ast

# this is saved as environment variable in the system.
openai.api_key = os.getenv("OPENAI_API_KEY_MICHAEL")

In [ ]:
# gold_rank_example = widgets.Dropdown(options=["[2, 3, 4, 1, 5]"], value="[2, 3, 4, 1, 5]", description='Gold Rank Example', layout={'height': 'auto', "width":"auto"})
# display(gold_rank_example)

insert_gold_rank_manually = widgets.Textarea(placeholder='Enter Gold Rank in the format of this [1,2,3,4,5]', value='', description='Gold Rank', rows=1, layout={'height': 'auto', "width":"auto"})
display(insert_gold_rank_manually)


prompt_identifier = widgets.Dropdown(options=["cot", "zero-shot", "one-shot-1","one-shot-2", "one-shot-3" , "two-shot"], value=None, description='Prompt Type Identifier', layout={'height': 'auto', "width":"auto"})
display(prompt_identifier)

text_input = widgets.Textarea(placeholder='Prompt', value='', description='Enter text:', rows=5, layout={'height': 'auto', "width":"auto"})
display(text_input)


# Creating widgets for the hyperparameters
model_input = widgets.Dropdown(options=["gpt-3.5-turbo", "gpt-4", "text-davinci-003"], value='gpt-3.5-turbo', description='model:', layout={'height': 'auto', "width":"auto"})
display(model_input)
temp_input = widgets.FloatSlider(value=0, min=0, max=1, step=0.05, description='temperature:', layout={'height': 'auto', "width":"auto"})
display(temp_input)
max_tokens_input = widgets.IntSlider(value=500, min=1, max=4000, step=1, description='max_tokens:', layout={'height': 'auto', "width":"auto"})
display(max_tokens_input)
top_p_input = widgets.FloatSlider(value=1.0, min=0, max=1, step=0.05, description='top_p:', layout={'height': 'auto', "width":"auto"})
display(top_p_input)
# freq_penalty_input = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.05, description='frequency_penalty:', layout={'height': 'auto', "width":"auto"})
# display(freq_penalty_input)
# presence_penalty_input = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.05, description='presence_penalty:', layout={'height': 'auto', "width":"auto"})
# display(presence_penalty_input)
# Creating the csv file if it does not exist and adding the new column "Hyperparameters used"


if not os.path.exists('response_eco.csv'):
    with open("response_eco.csv", "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Prompt With Task",
                         "Gold Rank",
                         "Prompt Type",
                         "Complete Response from Model",
                         "Output Only From Model",
                         "Hyperparameters used",
                         "Date and Time",
                        "Model"])

# Creating the function that will be called on button click

def on_submit(b):
    entered_text = text_input.value
    
    if model_input.value == "gpt-4" or model_input.value == "gpt-3.5-turbo":
        
        response = openai.ChatCompletion.create(
            model=model_input.value,
            messages=[{"role": "user", "content": entered_text}],
            # prompt=entered_text,
            temperature=temp_input.value,
            max_tokens=max_tokens_input.value,
            top_p=top_p_input.value,
            # frequency_penalty=freq_penalty_input.value,
            # presence_penalty=presence_penalty_input.value,
        )
        
        print("--------------------------------------------------------------------------------")
        print("Input: {}\nOutput: \n \n{}\n".format(entered_text, response['choices'][0]['message']['content']))

    elif model_input.value == "text-davinci-003":
        
        response = openai.Completion.create(
            model=model_input.value,
            prompt=entered_text,
            temperature=temp_input.value,
            max_tokens=max_tokens_input.value,
            top_p=top_p_input.value,
            # frequency_penalty=freq_penalty_input.value,
            # presence_penalty=presence_penalty_input.value,
        )
        print("--------------------------------------------------------------------------------")
        print("Input: {}\nOutput: \n \n{}\n".format(entered_text, response['choices'][0]["text"]))
        
    now = datetime.datetime.now()
    # Storing the hyperparameters used in the response
    params = {
        'model': model_input.value, 
        'temperature': temp_input.value, 
        'max_tokens': max_tokens_input.value, 
        'top_p': top_p_input.value, 
        # 'frequency_penalty': freq_penalty_input.value, 
        # 'presence_penalty': presence_penalty_input.value
    }
    
    
    if model_input.value == "gpt-4" or model_input.value == "gpt-3.5-turbo":
        print("I am in gpt-3.5-turbo")
        data = [
            entered_text,
            insert_gold_rank_manually.value,
            prompt_identifier.value,
            response,
            response['choices'][0]['message']['content'],
            params,
            now.strftime("%Y-%m-%d %H:%M:%S"),
            model_input.value
               ]
        
    elif model_input.value == "text-davinci-003":
        data = [
            entered_text,
            insert_gold_rank_manually.value,
            prompt_identifier.value,
            response,
            response['choices'][0]["text"], # for text-davinci-003
            # response['choices'][0]['message']['content'],
            params,
            now.strftime("%Y-%m-%d %H:%M:%S"),
            model_input.value
               ]
        
    # Open the CSV file for writing
    with open("response_eco.csv", "a", newline='') as file:
        writer = csv.writer(file)
        # Write the data rows
        writer.writerow(data)


submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit)
display(submit_button)

# gold rank
# [2, 3, 4, 1, 5]

In [ ]:
'''
response_eco has the complete response and everything else that we have got by prompting
the LLM
'''
df = pd.read_csv("response_eco.csv", encoding='cp1252')
print(df.iloc[11]['Prompt With Task'])

In [ ]:
print(df.iloc[-1]['Output Only From Model'])

In [ ]:
'''
Functionality:

Extract the rank from the model's output (which is saved in the csv file)
and create a new column containing (in the same df) the ranks in a list.
'''

# Define a function to extract the numbers associated with Rank
def extract_ranks(text):
    return [int(x) for x in re.findall(r'SAMPLE_TEXT_\d+: Rank (\d+)', text)]

# Apply the function to the column of the dataframe
df['model ranks'] = df['Output Only From Model'].apply(extract_ranks)
df


In [ ]:
'''
Functionality: converting the gold rank that is in string format
now to a list of integer

(If I have pre-processed it while saving the gold-rank then I would not
have to do it. With string you cannot calcualte spearman rank correlation)

'''


def str_to_list(str_list):
    # Remove the brackets from the string
    input_str = str_list[1:-1]

    # Split the string at the commas
    input_list = input_str.split(",")

    # Convert each element to an integer
    input_list = [int(x) for x in input_list]

    return input_list

df['Gold Rank'] = df['Gold Rank'].apply(str_to_list)
type(df.iloc[-1]['Gold Rank'])

In [ ]:
from scipy.stats import spearmanr
'''
For all the rank we got and create a another column with it
'''
def calc_spearman_rank_corr(row):
    gold_rank = row['Gold Rank']
    model_ranks = row['model ranks']
    corr, pvalue = spearmanr(gold_rank, model_ranks)
    print(corr)
    return corr

df['spearman_rank_correlation'] = df.apply(calc_spearman_rank_corr, axis=1)
df

In [ ]:
'''
Functionarlity:

Saving the final dataframe with all the ranks extracted and the spearman
rank correlation FINALLY!
'''

df.to_csv('complete_response_with_spearman.csv', index=False)

In [ ]:
sample_length = len(df)
average_correlation = statistics.mean(df['spearman_rank_correlation'])
variance = statistics.variance(df['spearman_rank_correlation'])
standard_deviation = statistics.stdev(df['spearman_rank_correlation'])
                                      
print("+++++++++++++++++++++++++++++")
print(f"Number of Samples: {sample_length}\nAverage Correlation: {average_correlation:.4f}\nVariance: {variance:.4f}\nStandardDeviation: {standard_deviation:.4f}")
print("+++++++++++++++++++++++++++++")

# Now using Local Model for ranking

In [ ]:
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.
which can be easily canged by chanigng the top_k parameter
"""
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Corpus with example sentences
corpus = ['Löhne werden regelmäßig erhöht. Jedoch werden die Löhne der Jobs, die eher keine Zukunft mehr haben niedriger ausfallen und wann will sie nicht mehr erhöhen. Dies wird aber von den Menschen als unfair bezeichnet, darum entstehen Streiks. Durch eine Inflation erhöhen sich die Lebenshaltungskosten, die Löhne bleiben aber bei diesen Jobs gleich. Hier führt es aber nicht zu Streiks, da es eine versteckte Jobentwertung ist. Somit sind die Löhne flexibler.',
          'Löhne sollten sich gemäß der Produktivität verändern. Eine positive Lohnveränderung kann aber nur bei einer positiven Produktionslücke stattfinden. Eine positive Produktionslücke bewirkt eine ansteigende Produktion. Maschinen verschleißen dann schneller und Produzenten müssen die Preise erhöhen. Dadurch steigt auch der Lohn, da der Preis einer Arbeitsstunde ebenfalls nach oben geht. Mit steigender Inflation können Löhne also flexibilisiert werden.',
          'insbesondere für kriselnde branchen und unternehmen birgt inflation finanzielle vorteile. denn durch diese sind („inflationsinfizierte“) lohnsenkungen möglich ohne das diese einen ähnlichen aufschrei der arbeitnehmer:innen zur folge hätte als eine herkömliche lohnsenkung. folglich sind mehr finanzielle mittel zur verfügung. des weiteren haben alle kreditnehmer:innen den vorteil, dass dieses relativ gesehen günstiger zurückzuzahlen ist.',
          'Aus einer Inflation können kurzfr. Steigerungen der Produktivität resultieren. Dies hat positive Auswirkungen auf die Löhne in den Branchen mit gesteigerter Produktivität. In Branchen mit verminderter Produktivität sollten eigentlich Lohnsenkungen das Resultat sein. Da diese als ungerecht empfunden werden bleiben diese aber oft aus. Somit führt die Inflation zu einer Senkung der Reallöhne bei gleich bleibenden nom. Löhnen und hat somit einen pos. Einfluss auf die Flexibilität der Löhne.',
          'Die Inflation beeinflusst die Löhne durch ihre Verzögerte Anpassung. Nehmen wir an die Löhne wären am Tag x festgelegt. Die Inflation steigt Zwischen X und Y um 10%. Da die Löhne Kollektiv verhandelt werden, kann eine Anpassung erst viel später folgen. Daraus Folgt dass die tatsächlichen Löhne gleich bleiben, das Unternehmen kann aber die Preise ihrer Produkte schneller anpassen. Ergo eine Gewinnsteigerung für Unternehmen.',
          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)


In [ ]:
corpus_embeddings

In [ ]:

# Query sentences:
queries = ['Löhne sollten mit erhöhter Produktivität eines Unternehmens steigen. Sinkt in einem Unternehmen die Produktivität, beispielsweise in sterbenden Branchen, so sollten in diesem die Löhne dementsprechend sinken. Eine solche Lohnsenkung wird jedoch von ArbeitnehmerInnen als unfair wahrgenommen und kann zu einer verringerten Arbeitsmoral und möglicherweise sogar zu Streiks führen. Bei Vorhandensein von Inflation kann eine Lohnsenkung dadurch erfolgen, dass Löhne nominal konstant bleiben und durch die Preissteigerung real entwertet werden. Dies wird oft als weniger unfair empfunden. So kann Inflation Unternehmen mehr Flexibilität geben, um eine unvermeidliche Lohnsenkung durchzuführen.']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    # print(query_embedding)
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    print(cos_scores)
    top_results = torch.topk(cos_scores, k=top_k, sorted=True)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))
        print("\n")

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

In [ ]:
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
print(cos_scores)
top_results = torch.topk(cos_scores, k=5, sorted=True)
top_results

In [ ]:
# Final Rank by local model
[2,1,5,3,4]



Explanation of how I arrived to the rank below from top_results.indices

indices=tensor([1, 0, 3, 4, 2])) <- list_K

The number that you can see in the indices are corresponding to the index of the corpus which has the 5 sentences.
Therefore, we see 4 in the above list, which means this is the index 4 of corpus list. And on index 4 we have this sentence
"Die Inflation beeinflusst die Löhne durch ihre Verzögerte Anpassung. Nehmen wir an die Löhne wären am Tag x festgelegt. Die Inflation steigt Zwischen X und Y um 10%. Da die Löhne Kollektiv verhandelt werden, kann eine Anpassung erst viel später folgen. Daraus Folgt dass die tatsächlichen Löhne gleich bleiben, das Unternehmen kann aber die Preise ihrer Produkte schneller anpassen. Ergo eine Gewinnsteigerung für Unternehmen."

This senence is in the index of 3 in the above list (list_K), which means the rank is 4 (I am assuming rank start from 1)

Another example.

We see 2 in the above list (list_K), wehich means this is the index 2 of the corpus list, and on index 2 we have this sentence:
"insbesondere für kriselnde branchen und unternehmen birgt inflation finanzielle vorteile. denn durch diese sind („inflationsinfizierte“) lohnsenkungen möglich ohne das diese einen ähnlichen aufschrei der arbeitnehmer:innen zur folge hätte als eine herkömliche lohnsenkung. folglich sind mehr finanzielle mittel zur verfügung. des weiteren haben alle kreditnehmer:innen den vorteil, dass dieses relativ gesehen günstiger zurückzuzahlen ist.'"

This sentence is in the index  of 4 in the above list (list_K) whihc means it is ranked as 5.








SAMPLE_TEXT_0 = RANK 2

SAMPLE_TEXT_1 = RANK 1

SAMPLE_TEXT_2 = RANK 5

SAMPLE_TEXT_3 = RANK 3

SAMPLE_TEXT_4 = RANK 4

In [ ]:
# Merging this new rank to the complete dataframe that has the rank from the three model

import pandas as pd

'''
Reading the complete response that has the spearman correlation also that I have
done previously. In here we do not have the info about the local model's rank
'''
df_temp = pd.read_csv("complete_response_with_spearman.csv")
df_temp

In [ ]:
from scipy.stats import spearmanr

'''
Calculating the sperman rank correlation with the ranking provided by
sentence transformer with the gold rank
'''

corr, pvalue = spearmanr([2, 3, 4, 1, 5], [2,1,5,3,4])
corr

In [ ]:
# from scipy.stats import spearmanr


# corr, pvalue = spearmanr([2, 3, 4, 1, 5], [2,4,3,1,5])
# corr

In [ ]:
import pandas as pd

# Assuming your existing DataFrame is called "df_temp"  

'''
Creating this dictionary that has the new information for the local model
to append it to the previous dataframe that has all the information except the local model's rank
'''

new_row = {"Model": "distiluse-base-multilingual-cased-v1",
           "model ranks": [2, 1, 5, 3, 4],
           "spearman_rank_correlation": 0.50,
           "Prompt Type": "zero-shot"}
# specifying inwhich location we should add the new item
df_temp.loc[len(df_temp)] = new_row
df_temp

In [ ]:
'''
Functionarlity:

Saving the final dataframe with all the ranks extracted and the spearman
rank correlation which also include the information of the local models rank
'''

df_temp.to_csv('complete_response_with_spearman_with_localmodel_also.csv', index=False)

In [ ]:
df_temp

# Extra Code

In [ ]:
"""USE THIS IF THE OUTPUT IS LIKE SHOWN BELOW TO EXTRACT THE RANKS: ACTUALLY YOU DO NOT
need this if you write this in the zero shot prompt
=========================================
Provide the rank in the following format

SAMPLE_TEXT_X: Rank X
==========================================
"""

# define the function to apply
def extract_numbers_from_template_one_shot_1_to_cot(text):
    '''
    This is exactly for "zero-shot" template because the output of the model is different like this
    
    1. SAMPLE_TEXT_4 
    2. SAMPLE_TEXT_1 
    3. SAMPLE_TEXT_6 
    4. SAMPLE_TEXT_2 
    5. SAMPLE_TEXT_5 
    6. SAMPLE_TEXT_3
    '''
    pattern = r"(?:Sample Text (\d+)|SAMPLE_TEXT_(\d+))$"
    numbers = re.findall(pattern, text)

    # Convert strings to integers
    numbers = [int(num1 or num2) for num1, num2 in re.findall(pattern, text, re.MULTILINE)]


    return numbers


df['model ranks'] = df.apply(lambda x: extract_numbers_from_template_one_shot_1_to_cot(x['Output']) if x['Prompt']=='zero-shot' else x['model ranks'], axis=1)
df